In [ ]:
import tensorflow as tf
from tensorflow import keras
import tensorflow_hub as hub

import numpy as np
import pandas as pd

In [ ]:
df = pd.read_csv("../input/imdb-dataset-of-50k-movie-reviews/IMDB Dataset.csv")

In [ ]:
df.loc[df["sentiment"] == "positive", "sentiment"] = 1
df.loc[df["sentiment"] == "negative", "sentiment"] = 0
df

In [ ]:
target = df.pop("sentiment")

In [ ]:
dataset = tf.data.Dataset.from_tensor_slices((df.values, target))

In [ ]:
train_val_data = dataset.take(25000)
test_data = dataset.skip(25000)

In [ ]:
train_val = train_val_data.take(10000) # 40% for validation
train_data = train_val_data.skip(10000) # 60% for training

In [ ]:
train_data_batch, train_label_batch = next(iter(train_data.batch(10)))
train_data_batch

In [ ]:
train_label_batch

In [ ]:
embedding = "https://tfhub.dev/google/tf2-preview/gnews-swivel-20dim/1"
model = keras.Sequential([
    hub.KerasLayer(embedding, input_shape=[], dtype=tf.string, trainable=True),
    keras.layers.Dense(16, activation='relu'),
    keras.layers.Dense(1)
])
model.summary()

In [ ]:
model.compile(optimizer='adam',
              loss=keras.losses.BinaryCrossentropy(from_logits=True),
              metrics=['accuracy'])

In [ ]:
train_data.shuffle(15000)
model.fit(x=tf.reshape(next(iter(train_data.batch(512)))[0], [-1,]),
          y=next(iter(train_data.batch(512)))[1],
          epochs=20,
          verbose=1)

In [ ]:
test_loss, test_acc = model.evaluate(x=tf.reshape(next(iter(test_data.batch(512)))[0], [-1,]),
                                     y=next(iter(test_data.batch(512)))[1],
                                     verbose=2)